# Convert custom data using PyNWB

Previously you learned how to convert electrophysiology data from a common data type (Neuropixels) to the NWB format, using the [GUIDE](https://nwb-guide.readthedocs.io/en/latest).

In this tutorial we'll learn how to use PyNWB to add custom trials data to an existing NWB file.

References:
- [PyNWB](https://pynwb.readthedocs.io/en/stable/index.html)

Let's start by installing the necessary packages.

In [ ]:
%%bash

pip install -U pynwb dandi

# Download NWB file from DANDI archive (staging)

In [1]:
from dandi.dandiapi import DandiAPIClient
import fsspec
import pynwb
import h5py
from fsspec.implementations.cached import CachingFileSystem
import requests


def download_file(url, target_path):
    try:
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(target_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=8192):
                    file.write(chunk)
            print(f"File downloaded successfully to {target_path}")
        else:
            print(f"Failed to download file. HTTP Status Code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")


def download_nwbfile(dandiset_id, filepath, token):
    api_kwargs = {
        "api_url": "https://api-staging.dandiarchive.org/api",
        "token": token
    }
    with DandiAPIClient(**api_kwargs) as client:
        asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
        download_url = asset.base_download_url

    download_file(url=download_url, target_path="downloaded_file.nwb")

In [2]:
download_nwbfile(
    dandiset_id='213840',
    filepath='sub-id0123/sub-id0123_ses-id987_ecephys.nwb',
    token="xxxxxxxxxxxxxxxxxxxxxxxxxx"
)

File downloaded successfully to downloaded_file.nwb


In [3]:
with pynwb.NWBHDF5IO("downloaded_file.nwb", mode="r+") as io:
    nwbfile = io.read()

    # create trials data and add them to the file
    nwbfile.add_trial_column(
        name="correct",
        description="whether the trial was correct",
    )
    nwbfile.add_trial(start_time=1.0, stop_time=5.0, correct=True)
    nwbfile.add_trial(start_time=6.0, stop_time=10.0, correct=False)

    io.write(nwbfile)

In [4]:
# read the modified NWB file
io = pynwb.NWBHDF5IO("downloaded_file.nwb", "r+")
nwbfile = io.read()

In [5]:
nwbfile

,location,group,group_name,channel_name,rel_y,contact_shapes,rel_x,shank_electrode_number,inter_sample_shift
id,,,,,,,,,
0,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x140652279222368\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x140652279223136\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP0,0.0,square,16.0,0.0,0.000000
1,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x140652279222368\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x140652279223136\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP1,0.0,square,48.0,1.0,0.000000
2,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x140652279222368\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x140652279223136\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP2,20.0,square,0.0,2.0,0.076923
3,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x140652279222368\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x140652279223136\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP3,20.0,square,32.0,3.0,0.076923
,location,group,group_name,channel_name,rel_y,contact_shapes,rel_x,shank_electrode_number,inter_sample_shift
id,,,,,,,,,
0,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x140652279222368\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x140652279223136\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP0,0.0,square,16.0,0.0,0.000000
1,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x140652279222368\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x140652279223136\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP1,0.0,square,48.0,1.0,0.000000
2,unknown,"s0 pynwb.ecephys.ElectrodeGroup at 0x140652279222368\nFields:\n description: a group representing shank s0\n device: Neuropixel-Imec pynwb.device.Device at 0x140652279223136\nFields:\n description: {""probe_type"": ""0"", ""probe_type_description"": ""NP1.0"", ""flex_part_number"": ""NP2_FLEX_0"", ""connected_base_station_part_number"": ""NP2_QBSC_00""}\n manufacturer: Imec\n\n location: unknown\n",s0,AP2,20.0,square,0.0,2.0,0.076923


In [ ]:
from dandi.download import download

download(
    urls="https://api-staging.dandiarchive.org/api/assets/9e6c25a9-2e46-4ea6-9375-37d2639c219b/download/",
    output_dir="."
)